In [1]:
from lasagne.layers import InputLayer, Conv2DLayer, MaxPool2DLayer, DenseLayer, GlobalPoolLayer, Upscale2DLayer
from lasagne.layers import ElemwiseSumLayer, NonlinearityLayer, SliceLayer, ConcatLayer, ScaleLayer
from lasagne.layers import dropout, batch_norm
from lasagne.nonlinearities import rectify, softmax, sigmoid
from lasagne.init import GlorotNormal, GlorotUniform, HeUniform, HeNormal
from lasagne.objectives import squared_error, categorical_crossentropy, categorical_accuracy, binary_accuracy
import lasagne
import theano.tensor as T
import numpy as np
import random
import theano
import os
import pandas as pd
import cv2
import re
import sys

sys.path.append('..')
import benchmark as bm

from fcn1.adapter import adapter as adapter1

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 3007)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


# Utility functions

In [2]:
def build_volume(area, location, resolution):
    def _step(idx_, prior_result_, area_, location_, resolution_):
        area1 = area_[idx_][0] * T.prod(resolution_[idx_])
        area2 = area_[idx_ + 1][0] * T.prod(resolution_[idx_ + 1])
        h = location_[idx_ + 1] - location_[idx_]
        volume = (area1 + area2 )* np.prod(fixed_size).astype('float32') * h / 2.0 / 1000
        return prior_result_ + volume

    predict_V_list, _ = theano.scan(fn=_step,
                              outputs_info = np.array([0.]).astype('float32'),
                              sequences = T.arange(1000),
                              non_sequences = [area, location, resolution],
                              n_steps = location.shape[0] - 1)
    predict_V = predict_V_list[-1]
    return predict_V[0]

def stage3_load_batch_records(file_path, fixed_size):
    data = np.load(file_path).item()
    patch_list = data['patchStack']
    location_list = np.array(data['SliceLocation'])
    resolution = np.array(data['PixelSpacing'])
    resized_resolution_list = []
    resized_patch_list = []
    for patch in patch_list:
        resized_resolution_list.append(
            (resolution[0] / fixed_size[0] * patch.shape[0], resolution[1] / fixed_size[1] * patch.shape[1]))
        resized_patch_list.append(cv2.resize(patch, fixed_size))

    resized_patch_list = np.array(resized_patch_list, dtype='float32')[:, None, :, :]
    location_list = np.array(location_list, dtype='float32')
    resized_resolution_list = np.array(resized_resolution_list, dtype='float32')
    
    patch_batch = [resized_patch_list, resized_patch_list[:,:,::-1,:],
                      resized_patch_list[:,:,:,::-1], resized_patch_list[:,:,::-1,::-1]]
    location_batch = [location_list] * 4
    resolution_batch = [resized_resolution_list] * 4
    return patch_batch, location_batch, resolution_batch

# Configuration

In [3]:
lr = 0.01
snapshot_root = 'snapshot'
snapshot_file = '0.npz'
snapshot_iter = 1
schedules = [2] * 3005
fixed_size = (48, 48)

# Load data

In [4]:
train_val_ratio = 0.8

# read train and val volume data
volume_csv_path = '../../clean/stage3/train.csv'
volume_csv = pd.read_csv(volume_csv_path)
volume_data = np.array(volume_csv.iloc[:, 1:3])
rows = volume_data.shape[0]
volume_data = np.repeat(volume_data.flatten(), 4).astype('float32')

# read train and val patch, location and resolution
root_dir = '../../clean/stage3'
min_root_dir = os.path.join(root_dir, 'min')
max_root_dir = os.path.join(root_dir, 'max')
x_data = []
location_data = []
resolution_data = []
for i in range(1, rows + 1):
    min_full_path = os.path.join(min_root_dir, str(i) + '.npy')
    max_full_path = os.path.join(max_root_dir, str(i) + '.npy')
    paths = [min_full_path, max_full_path]
    for path in paths:
        x_data_batch, location_data_batch, resolution_data_batch = stage3_load_batch_records(path, fixed_size)
        x_data.extend(x_data_batch)
        location_data.extend(location_data_batch)
        resolution_data.extend(resolution_data_batch)
# fixed seperation, for comparing different models
n_samples_train = int(len(x_data) * train_val_ratio)
n_samples_val = len(x_data) - n_samples_train

# split data
x_data_train = x_data[:n_samples_train]
location_data_train = location_data[:n_samples_train]
resolution_data_train = resolution_data[:n_samples_train]
volume_data_train = volume_data[:n_samples_train]
x_data_val = x_data[n_samples_train:]
location_data_val = location_data[n_samples_train:]
resolution_data_val = resolution_data[n_samples_train:]
volume_data_val = volume_data[n_samples_train:]

n_train_batches = n_samples_train
n_val_batches = n_samples_val

print("------------data info---------------------")
print("TRAIN, x: {}, location: {}, resolution: {}, volume: {}".format(len(x_data_train), len(location_data_train),
                                                                     len(resolution_data_train), len(volume_data_train)))
print("VAL, x: {}, location: {}, resolution: {}, volume: {}".format(len(x_data_val), len(location_data_val),
                                                                     len(resolution_data_val), len(volume_data_val)))

------------data info---------------------
TRAIN, x: 3200, location: 3200, resolution: 3200, volume: 3200
VAL, x: 800, location: 800, resolution: 800, volume: 800


# Build model

In [5]:
# x = T.tensor4('x')
location = T.vector('location')
resolution = T.matrix('resolution')
target_volume = T.fscalar('volume')

# all adapters
adapters = []
adapters.append(adapter1((64, 64), '60.npz'))

# input tensor
pred = T.tensor4('pred')

# fusion layers
l_in = InputLayer(shape=(None, len(adapters), fixed_size[0], fixed_size[1]), input_var = pred)
mid = batch_norm(Conv2DLayer(l_in, num_filters=32, filter_size=(3, 3), nonlinearity=rectify, W=HeNormal()))
l_out = GlobalPoolLayer(Conv2DLayer(mid, num_filters=1, filter_size=(1, 1), W=HeNormal()))

area = lasagne.layers.get_output(l_out)
pred_volume = build_volume(area, location, resolution)
loss = T.abs_(pred_volume - target_volume).mean() / 600

params = lasagne.layers.get_all_params(l_out, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=lr, momentum=0.9)

train_fn = theano.function(
    [pred, location, resolution, target_volume],
    loss,
    updates = updates
)

val_fn = theano.function(
    [pred, location, resolution, target_volume],
    loss
)

test_fn = theano.function(
    [pred, location, resolution],
    [area, pred_volume]
)

# Load snapshot

In [6]:
print("-----------snapshot info--------------------")
cur_iter = 0
if snapshot_root is not None:
    if not os.path.exists(snapshot_root):
        print("creating {}".format(snapshot_root))
        os.mkdir(snapshot_root)
    if snapshot_file is not None: 
        snapshot_full_path = os.path.join(snapshot_root, snapshot_file)
        if os.path.exists(snapshot_full_path):
            with np.load(snapshot_full_path) as f:
                param_values = [f['arr_{}'.format(i)] for i in range(len(f.files))]
            print('resuming snapshot from {}'.format(snapshot_full_path))
            param_cur = lasagne.layers.get_all_params(l_out)
            assert len(param_cur) == len(param_values)
            for p, v in zip(param_cur, param_values):
                p.set_value(v)
            m = re.findall('\d+', snapshot_file)
            if len(m) > 0:
                cur_iter = int(m[0])
        else:
            print("snapshot file {} is not exist".format(snapshot_full_path))
print("cur_iter: {}".format(cur_iter))

-----------snapshot info--------------------
snapshot file snapshot/0.npz is not exist
cur_iter: 0


# Training and Validating

In [ ]:
print("-----------train and validation info------------")
if schedules is not None:
    for n_epoches in schedules:
        print("#epochs: {}， lr: {}".format(n_epoches, lr))
        while n_epoches > 0:
            cur_iter += 1
            residual_epoch = min(n_epoches, 1)
            seen_train_samples = int(n_train_batches * residual_epoch)
            train_idx = np.random.permutation(n_train_batches)[:seen_train_samples]
            val_idx = np.random.permutation(n_val_batches)
            train_losses = []
            train_accs = []
            val_losses = []
            val_accs = []
            for j in train_idx:
                x_e = x_data_train[j].astype('float32')
                preds = []
                for adapter in adapters:
                    preds.append(adapter.convert(x_e))
                pred_e = np.concatenate(preds, axis=1)
                location_e = location_data_train[j].astype('float32')
                resolution_e = resolution_data_train[j].astype('float32')
                volume_e = volume_data_train[j].astype('float32')
                loss = train_fn(pred_e, location_e, resolution_e, volume_e)
                train_losses.append(loss)
            for j in val_idx:
                x_e = x_data_val[j].astype('float32')
                preds = []
                for adapter in adapters:
                    preds.append(adapter.convert(x_e))
                pred_e = np.concatenate(preds, axis=1)
                location_e = location_data_val[j].astype('float32')
                resolution_e = resolution_data_val[j].astype('float32')
                volume_e = volume_data_val[j].astype('float32')
                loss = val_fn(pred_e, location_e, resolution_e, volume_e)
                val_losses.append(loss)
            print("stage3: train loss {}[seen {} samples], val loss {}[seen {} samples][{}]"\
                  .format(np.mean(train_losses), seen_train_samples, 
                          np.mean(val_losses), n_val_batches, cur_iter))
            n_epoches -= 1
            if cur_iter % snapshot_iter == 0:
                snapshot_save_path = os.path.join(snapshot_root, str(cur_iter) + ".npz")
                print("snapshot to {}".format(snapshot_save_path))
                np.savez(snapshot_save_path, *lasagne.layers.get_all_param_values(l_out))
else:
    print("schedules is empty, skip training and validating stages")

In [ ]:
bm.test_on_train_data(test_fn, fixed_size)